<a href="https://colab.research.google.com/github/srinivasiyengar/srinivas/blob/master/KDAG_srinivas_iynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd 

In [2]:
df = pd.read_csv('/train.csv') 
df = df.drop(['A','id'], axis=1)

In [3]:
df_test = pd.read_csv('/test.csv')
df_test1 = df_test.drop(['A','id'], axis=1)

In [4]:
for column in df.columns:
  if df[column].dtypes == object:
    for cat in df[column].unique():
      if(cat not in df_test1[column].unique()):
        indexNames = df[df[column]==cat].index
        df.drop(indexNames , inplace=True)

In [5]:
from sklearn import preprocessing 
le = preprocessing.LabelEncoder() 
for column in df.columns:
  if df[column].dtypes != int:
    df[column] = le.fit_transform(df[column])

In [6]:
X = df.drop(['poor'], axis=1)
Y = df["poor"]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [8]:
from xgboost import XGBClassifier
model = XGBClassifier()

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=4, random_state=0, algorithm='SAMME')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model= RandomForestClassifier()

In [ ]:
from lightgbm import LGBMClassifier
model = lightgbm.LGBMClassifier(boosting_type='gbdt', num_leaves=100, max_depth=- 1, learning_rate=0.4, n_estimators=100, subsample_for_bin=200000, objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, silent=True, importance_type='split', **kwargs)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=350, learning_rate=0.5, max_features=25, max_depth=1, random_state=0)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.calibration import CalibratedClassifierCV

calibrated_forest = CalibratedClassifierCV(base_estimator=RandomForestClassifier(n_estimators=10))
param_grid = {'base_estimator__max_depth': [2, 4, 6, 8]}
search = GridSearchCV(calibrated_forest, param_grid, cv=5)
search.fit(X_train, Y_train)

In [ ]:
from sklearn.ensemble import BaggingClassifier
seed = 8
num_trees = 100
boosted_clf = BaggingClassifier(base_estimator = model, 
                          n_estimators = num_trees, 
                          random_state = seed) 

In [ ]:
boosted_clf.fit(X_train, Y_train)

In [9]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
sfs1 = sfs(model,
           k_features=50,
           forward=True,
           floating=False,
           verbose=2,
           scoring='accuracy',
           cv=5)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
sfs1 = sfs1.fit(X_train, Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 333 out of 333 | elapsed:  2.4min finished

[2020-10-22 07:56:02] Features: 1/50 -- score: 0.7045893972074015[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 332 out of 332 | elapsed:  3.2min finished

[2020-10-22 07:59:11] Features: 2/50 -- score: 0.730166420706096[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 331 out of 331 | elapsed:  3.9min finished

[2020-10-22 08:03:06] Features: 3/50 -- score: 0.7740077950580845[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   

In [21]:
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)

[24, 28, 176, 276, 299]


In [ ]:
model.fit(X_train, Y_train)

In [ ]:
Y_pred = model.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.8886569241475296

In [ ]:
x = df_test1

In [ ]:
from sklearn import preprocessing 
le = preprocessing.LabelEncoder() 
for column in x.columns:
  if x[column].dtypes != int:
    x[column] = le.fit_transform(x[column])

In [ ]:
y_pred = model.predict(x)
y_pred = pd.DataFrame(y_pred)

In [ ]:
y_pred

,0
0,1
1,0
2,0
3,1
4,1
...,...
816,0
817,1
818,0
819,0


In [ ]:
data = pd.concat([df_test['id'], y_pred], axis=1, sort=False)
data

,id,0
0,93441,1
1,25393,0
2,10833,0
3,29837,1
4,79778,1
...,...,...
816,47918,0
817,56855,1
818,74878,0
819,77513,0


In [ ]:
data.to_csv('kdag5.csv')